# Multistage Stochastic Unit Commitment Problem

In [1]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg
from itertools import product


from sddip import config, utils, tree


## Parameters

In [2]:
test_case_raw_dir = "case6ww/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\s+")
branch_df = pd.read_csv(branch_file_raw, delimiter="\s+")
gen_df = pd.read_csv(gen_file_raw, delimiter="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\s+")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\s+")

In [3]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

ptdf = np.insert(ptdf, ref_bus-1, 0, axis=1)

In [4]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)

# TODO Adjust penalty
penalty = 10000
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_realizations_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_realizations_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [5]:
# TODO min up-/down-time
min_up_time = [2]*n_gens
min_down_time = [2]*n_gens

In [6]:
gens_at_bus

[[0], [1], [2], [], [], []]

In [7]:
# prob[t][n]
# Probability of realization n at stage t
#prob

In [8]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
#p_d

In [9]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
#ex_pd

In [10]:
scenario_tree = tree.ScenarioTree(n_realizations_per_stage)
print(scenario_tree)

ScenarioTree: Stages = 12, Nodes = 4095


## Optimization

### Variables

In [11]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}

# TODO Set startup and shutdown costs
# suc = [10]*n_gens
# sdc = [10]*n_gens

for t in range(n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-29


### Constraints

In [12]:
# Objective
obj = gp.quicksum(1/scenario_tree.n_nodes_per_stage[t]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g] + penalty*(ys_p[t,n] + ys_n[t,n])) 
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])
                    for g in range(n_gens))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n.index,g] for g in range(n_gens)) + ys_p[t,n.index] - ys_n[t,n.index] == gp.quicksum(p_d[t][n.realization])
                    for t in range(n_stages)
                    for n in scenario_tree.get_stage_nodes(t)),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g]*x[t,n,g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g]*x[t,n,g]  
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-generation")


# Power flow constraints
for t in range(n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) - p_d[t][node.realization][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] == s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((x[t,n,g] - x[t-1,a_n,g] <= s_up[t,n,g] 
                        for g in range(n_gens)), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,a_n,g] == s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((y[t,n,g] - y[t-1,a_n,g] <= rg_up_max[g]
                        for g in range(n_gens)), 
                        "rate-up")
        model.addConstrs((y[t-1,a_n,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)), 
                        "rate-down")


# Minimum up- and down-time constraints
for g in range(n_gens):
    for t in range(1, min_up_time[g]):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors()
            model.addConstr((gp.quicksum(x[m.stage,m.index,g] for m in ancestors) >= (t+1)*s_down[t,n,g]), "min-uptime")
    
    for t in range(min_up_time[g], n_stages):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors(min_up_time[g])
            model.addConstr((gp.quicksum(x[m.stage, m.index,g] for m in ancestors) >= min_up_time[g]*s_down[t,n,g]), "min-uptime")

    for t in range(1, min_down_time[g]):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors()
            model.addConstr((gp.quicksum((1-x[m.stage,m.index,g]) for m in ancestors) >= (t+1)*s_up[t,n,g]), "min-downtime")

    for t in range(min_down_time[g], n_stages):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors(min_down_time[g])
            model.addConstr((gp.quicksum((1-x[m.stage,m.index,g]) for m in ancestors) >= min_down_time[g]*s_up[t,n,g]), "min-downtime")


model.update()
#model.display()

### Solve

In [13]:
model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")
print()
print(f"Optimal value: {obj.getValue()}")


Optimal value: 21202.634616017443


In [14]:
display_results = False

if display_results:
    x_out = [f"x[{t+1},{n+1},{g+1}]:  {x[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    y_out = [f"y[{t+1},{n+1},{g+1}]:  {y[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    s_up_out = [f"s_up[{t+1},{n+1},{g+1}]:  {s_up[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    s_down_out = [f"s_down[{t+1},{n+1},{g+1}]:  {s_down[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]

    print("Commitment decisions")
    for text in x_out:
        print(f"{text}")

    print("Dispatch decisions")
    for text in y_out:
        print(f"{text}")

    print("Startup decisions")
    for text in s_up_out:
        print(f"{text}")

    print("Shutdown decisions")
    for text in s_down_out:
        print(f"{text}")

In [15]:
costs = []
c = 0
for t in reversed(range(n_stages)):
    for n in range(scenario_tree.n_nodes_per_stage[t]):
        for g in range(n_gens):
            c += 1/scenario_tree.n_nodes_per_stage[t] * y[t,n,g].x*gc[g]
    costs.append(c)


costs

[1744.9038054575242,
 3981.206207675645,
 6105.71056784157,
 7738.676124137741,
 9488.760855720844,
 11498.92434148991,
 13449.059868452488,
 15408.261032991302,
 16950.676874400073,
 17694.388695702582,
 18345.988782978937,
 19202.634616010837]

In [16]:
#model.display()